In [0]:
try:
    import transformers
    import apex
    from easydict import EasyDict as edict
    import sinkhorn_transformer
    #import torch_xla
except ImportError as e:
    !pip install transformers
    !pip install easydict
    !git clone https://github.com/NVIDIA/apex
    %cd apex
    !pip install -v  --global-option="--cpp_ext" --global-option="--cuda_ext" --no-cache-dir ./
    %cd ..
    !pip install sinkhorn_transformer
    !pip install tokenizers==0.6.0
    #VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
    #!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    #!python pytorch-xla-env-setup.py --version $VERSION

     |████████████████████████████████| 645kB 5.7MB/s 
     |████████████████████████████████| 3.8MB 16.7MB/s 
     |████████████████████████████████| 1.0MB 39.7MB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=64f08e962808c1a394149610117c1c6347a34e01ffe64d55120dbe7945d98918
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'apex'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 6733 (delta 91), reused 77 (delta 49), pack-reused 6596
Receiving objects: 100% (6733/6733), 13.76 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (4479/4479), done.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My\ Drive/Master/TFM

[Errno 2] No such file or directory: 'drive/My Drive/TFM'
/content


In [4]:
%load_ext autoreload
%autoreload 2

from generator_utils import predict_beam, predict_top_p, predict_top_k
from data_utils import create_dataloader, get_processed_files_splited
from model_utils import get_config, get_model, get_tokenizer, save_all
from train_utils import train, evaluate
from vocabulary import Vocabulary
from easydict import EasyDict as edict
import torch

#import torch_xla
#import torch_xla.core.xla_model as xm

import pickle
import torch
import os

ModuleNotFoundError: ignored

In [0]:
# List of transformers models: https://huggingface.co/transformers/pretrained_models.html
args = {
    'repositories_path': '/home/manuelj/data/TFM/',
    #'vocabulary_path': './python_tokenizers/python_tokenizer_flask',
    'vocabulary_path': './BPE_tokenizers/python-bpe-django',
    'sinkhorn_bucket_size': 20,
    'main_file_name': 'processed.txt',
    'vocab_size': 0,
    'model_name': 'gpt2',#gpt2',#'xlnet-base-cased', 
    'seq_size': 100,
    'batch_size': 16,
    'epochs': 100,
    'lr': 5e-5,
    'epsilon': 1e-8,
    'weight_decay': 0.0,
    'opt_level': 'O1',
    'gradient_accumulation_steps': 1,
    'warmup_steps': 0,
    'output_dir': './checkpoints_gpt2',
    'checkpoint_model': False,
    'restore_training': False,
    'limit': None,
    'save_steps': 5,
    'train_len': None,#15332, 21648
    'use_python_vocabulary': False,
    'out_directory': './results/flask/model_gpt2_flask_bpe_tokenizer_100e',#'./results/django/model_gpt2_django_bpe_tokenizer_10e',
    'fp16': True,
    'tpu':False,
    'show_logs_during_train': False,
    'train_logs_freq': 100,
    'transformers_models': ['bert', 'gpt2', 'xlnet'],
    'data_step': 100
}
args = edict(args)

In [0]:
if args.use_python_vocabulary:
    vocabulary = Vocabulary()
    vocabulary.load_vocab(args.vocabulary_path)
    args.vocab_size =  len(vocabulary.word2index.keys())
else:
    vocabulary = get_tokenizer(args.vocabulary_path)
    args.vocab_size =  vocabulary.get_vocab_size()

Vocab File: ./BPE_tokenizers/python-bpe-flask-vocab.json
Merges file: ./BPE_tokenizers/python-bpe-flask-merges.txt


In [0]:
data_path = args.repositories_path
fileplath = args.main_file_name
train_files, valid_files, test_files = get_processed_files_splited(train_path='./train_files_flask', valid_path='./valid_files_flask', test_path='./test_files_flask', directory=True)
if args.tpu:
  device = xm.xla_device()
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
train_dataloader = create_dataloader(args, train_files, vocabulary)
val_dataloader = create_dataloader(args, valid_files, vocabulary)

In [0]:
model = get_model(args, device)
model = model.to(device)

In [0]:
model, results = train(train_dataloader, model, device, args, eval_dataloader=val_dataloader)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Train length: 226
***** Running training *****



Time: 2.1048011978467307 Train Loss: 8.454196123950249 Train Acc: 0.05915817618370056 Train Acc@5: 0.113723985850811 Train PP: 4694.73046875 | Eval Loss: 7.488679866790772 Eval Acc: 0.12361069768667221 Eval Acc@5: 0.23846431076526642 Eval PP: 71.50762176513672



Time: 4.21886127392451 Train Loss: 7.567447932420579 Train Acc: 0.12198563665151596 Train Acc@5: 0.20735621452331543 Train PP: 1934.1981201171875 | Eval Loss: 6.8774324226379395 Eval Acc: 0.17007499933242798 Eval Acc@5: 0.30641427636146545 Eval PP: 38.8052864074707


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0




Time: 6.336783691247305 Train Loss: 7.138181711720154 Train Acc: 0.15519796311855316 Train Acc@5: 0.2551177740097046 Train PP: 1259.1368408203125 | Eval Loss: 6.366762580871582 Eval Acc: 0.1909106820821762 Eval Acc@5: 0.3516499996185303 Eval PP: 23.28680419921875


...


Time: 189.23152878284455 Train Loss: nan Train Acc: 0.0 Train Acc@5: 6.91371678840369e-05 Train PP: nan | Eval Loss: nan Eval Acc: 0.0 Eval Acc@5: 7.499999628635123e-05 Eval PP: nan


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 3.6260443998244473e-217
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1.8130221999122236e-217
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 9.065110999561118e-218
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4.532555499780559e-218
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2.2662777498902796e-218
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1.1331388749451398e-218
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 5.665694374725699e-219
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2.8328471873628494e-219
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1.4164235936814247e-219
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 7.082117968407124e-220
Gradient overflow.  Skipping step, loss scal


Time: 191.1693101366361 Train Loss: nan Train Acc: 0.0 Train Acc@5: 6.91371678840369e-05 Train PP: nan | Eval Loss: nan Eval Acc: 0.0 Eval Acc@5: 7.499999628635123e-05 Eval PP: nan


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 3.36243654762363e-285
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1.681218273811815e-285
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8.406091369059075e-286
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4.2030456845295373e-286
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2.1015228422647686e-286
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1.0507614211323843e-286
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 5.253807105661922e-287
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2.626903552830961e-287
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1.3134517764154804e-287
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 6.567258882077402e-288
Gradient overflow.  Skipping step, loss scaler 

ZeroDivisionError: ignored

In [0]:
save_all(args, model, results)

Saved!


In [0]:
model = load_model(args)
model = model.to(device)

<All keys matched successfully>

In [0]:
text = """def basic_pages(**kwargs):
    return make_response(open('angular_flask/templates/index.html').read())
from sqlalchemy.sql import exists
crud_url_models = app.config['CRUD_URL_MODELS']
@app.route('/<model_name>/')
@app.route('/<model_name>/<item_id>')
"""

In [0]:
text="""import unittest
from datetime import datetime
from werkzeug.testsuite import WerkzeugTestCase
from werkzeug import http, datastructures
from werkzeug.test import create_environ
class HTTPUtilityTestCase(WerkzeugTestCase):
  def test_accept(self):
    a = http.parse_accept_header('en-us,ru;q=0.5')
    self.assert_equal(a.values(), ['en-us', 'ru'])
    self.assert_equal(a.best, 'en-us')
    """

In [0]:
text="""from datetime import datetime
from flask_wtf import Form, RecaptchaField
from wtforms import (StringField, PasswordField, BooleanField, HiddenField,
                     SubmitField, SelectField)
from wtforms.validators import (DataRequired, InputRequired, Email, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User
USERNAME_RE = r'^[\w.+-]+$'
is_username = regexp(USERNAME_RE,
                     message=_("You can only use letters, numbers or dashes."))
class LoginForm(Form):
    login = StringField(_("Username or E-Mail Address"), validators=["""

In [0]:

predict_beam(args, device, model, text, vocabulary, n_out=1)

100
100
, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User
USERNAME_RE = r'^[\w.+-]+$'
is_username = regexp(USERNAME_RE,
                     message=_("You can only use letters, numbers or dashes."))
class LoginForm(Form):
    login = StringField(_("Username or E-Mail Address"), validators=[log
------------------------------
, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User
USERNAME_RE = r'^[\w.+-]+$'
is_username = regexp(USERNAME_RE,
                     message=_("You can only use letters, numbers or dashes."))
class LoginForm(Form):
    login = StringField(_("Username or E-Mail Address"), validators=[stuff
------------------------------
, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User

In [0]:
predict_top_p(args, device, model, text, vocabulary, n_out=1)

, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User
USERNAME_RE = r'^[\w.+-]+$'
is_username = regexp(USERNAME_RE,
                     message=_("You can only use letters, numbers or dashes."))
class LoginForm(Form):
    login = StringField(_("Username or E-Mail Address"), validators=[�
------------------------------
, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User
USERNAME_RE = r'^[\w.+-]+$'
is_username = regexp(USERNAME_RE,
                     message=_("You can only use letters, numbers or dashes."))
class LoginForm(Form):
    login = StringField(_("Username or E-Mail Address"), validators=[ be
------------------------------
, EqualTo,
                                regexp, ValidationError)
from flask_babelplus import lazy_gettext as _
from flaskbb.user.models import User
USERNAME_RE

In [0]:
predict_top_k(args, device, model, text, vocabulary, n_out=1)

100
 op
import sqlalchemy as sa
def upgrade():
    op.create_table('posts',
    sa.Column('id', sa.Integer(), nullable=False),
    sa.Column('body', sa.Text(), nullable=True),
    sa.Column('author_id', sa.Integer(), nullable=True),
    sa.ForeignKeyConstraint(['author_id'], ['users.id'], ),
    sa.PrimaryKeyConstraint('id')
    )
    ',
------------------------------
 op
import sqlalchemy as sa
def upgrade():
    op.create_table('posts',
    sa.Column('id', sa.Integer(), nullable=False),
    sa.Column('body', sa.Text(), nullable=True),
    sa.Column('author_id', sa.Integer(), nullable=True),
    sa.ForeignKeyConstraint(['author_id'], ['users.id'], ),
    sa.PrimaryKeyConstraint('id')
    )
    ()
------------------------------
 op
import sqlalchemy as sa
def upgrade():
    op.create_table('posts',
    sa.Column('id', sa.Integer(), nullable=False),
    sa.Column('body', sa.Text(), nullable=True),
    sa.Column('author_id', sa.Integer(), nullable=True),
    sa.ForeignKeyConstraint(['aut